In [ ]:
!nvidia-smi

Sun Sep 25 13:22:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install comet-ml
!pip install scikit-multilearn
!pip install transformers 
!pip install datasets
!pip install nlp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.22.1-py3-none-any.whl (4.9 MB)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!unzip Вологда.zip

Archive:  Вологда.zip
Вологда/:  mismatching "local" filename (╨Т╨╛╨╗╨╛╨│╨┤╨░/),
         continuing with "central" filename version
   creating: Вологда/
Вологда/.ipynb_checkpoints/:  mismatching "local" filename (╨Т╨╛╨╗╨╛╨│╨┤╨░/.ipynb_checkpoints/),
         continuing with "central" filename version
   creating: Вологда/.ipynb_checkpoints/
Вологда/.ipynb_checkpoints/vologda_baseline-checkpoint.ipynb:  mismatching "local" filename (╨Т╨╛╨╗╨╛╨│╨┤╨░/.ipynb_checkpoints/vologda_baseline-checkpoint.ipynb),
         continuing with "central" filename version
  inflating: Вологда/.ipynb_checkpoints/vologda_baseline-checkpoint.ipynb  
Вологда/sample_solution.csv:  mismatching "local" filename (╨Т╨╛╨╗╨╛╨│╨┤╨░/sample_solution.csv),
         continuing with "central" filename version
  inflating: Вологда/sample_solution.csv  
Вологда/test_dataset_test.csv:  mismatching "local" filename (╨Т╨╛╨╗╨╛╨│╨┤╨░/test_dataset_test.csv),
         continuing with "central" filename version
  inflating: Вологд

In [ ]:
import comet_ml
comet_ml.init(project_name='vologda')

import torch
from pathlib import Path
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModel, AutoModelForSequenceClassification,AutoModelForTokenClassification,TrainingArguments, Trainer
from tqdm.auto import tqdm
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import log_loss
from sklearn.preprocessing import MultiLabelBinarizer

np.random.seed(1337)
torch.manual_seed(1337)

COMET INFO: Comet API key is valid


In [ ]:
from transformers import BertForMaskedLM,BertTokenizerFast, pipeline, BatchEncoding,BertForTokenClassification,BertForSequenceClassification
from tokenizers import Encoding
model_ckpt = "sberbank-ai/ruRoberta-large" 
#model_ckpt = "sberbank-ai/ruBert-large" 
#model_ckpt = "yandex/RuLeanALBERT"
#model_ckpt = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")

In [ ]:
df_train = pd.read_csv('Вологда/train_dataset_train.csv')

In [ ]:
df_train["len_reviews"] = df_train["Отзыв"].map(len)

In [ ]:
train = df_train.copy().drop(columns = 'RecordNo',inplace = False)
train.fillna(0)

,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,len_reviews
0,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0,3,0,0,0,0,0,157
1,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0,1,0,0,0,0,1,283
2,Дети мои,Гузель Яхина,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0,0,0,1,0,0,1,278
3,ПОСТ,Дмитрий Глуховский,https://www.litres.ru/dmitriy-gluhovskiy/post/,3.9,430,48,Kimetov Oleg,5.0,Очень понравилось. Причем понравилось больше «...,2,7,0,1,0,0,0,342
4,ПОСТ,Дмитрий Глуховский,https://www.litres.ru/dmitriy-gluhovskiy/post-...,4.4,832,113,Сергей Никитин,5.0,Круто. Автор и он же чтец – гений. Прослушал н...,1,0,0,0,0,0,0,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4834,Дети мои,Гузель Яхина,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,417504557,4.0,Ещё один замечательный роман Гузель Яхиной! По...,0,0,0,0,0,0,0,149
4835,Авиатор,Евгений Водолазкин,https://www.litres.ru/evgeniy-vodolazkin/aviator/,4.5,3551,260,Viktoria Steinbach,0.0,Как всегда у Водолазкина – увлекательнейший сю...,0,1,1,1,0,0,1,468
4836,Текст,Дмитрий Глуховский,https://www.litres.ru/dmitriy-gluhovskiy/tekst...,4.5,1923,246,AnVer,5.0,"Мрачно, грустно. Давно такого не читала и хват...",1,2,0,0,1,0,0,148
4837,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Мария Перешивко,5.0,Очень интересная книга! Как будто очутилась в ...,0,1,0,0,0,0,0,139


In [ ]:
cat_features = ['Название книги','Автор','Имя читателя','Рейтинг']
text_features = ['Отзыв']
target_columns = ['Релевантность',
       'Таксономия релевантные', 'Таксономия не релевантные',
       'Длина отзыва', 'Ценности']

train_labels = train.copy()[target_columns].values
train_data  = train.copy().drop(columns = target_columns,inplace = False)
train_data[cat_features] = train_data[cat_features].astype(str)
train_data[text_features] = train_data[text_features].astype(str)

In [ ]:
def create_text_from_cols(df):
  texts = pd.DataFrame()
  texts['text'] = [("Автор: " + str(t['Автор']) + ' ' + "Имя читателя: " + str(t['Имя читателя']) + ' ' +  'Отзыв: ' + str(t['Отзыв']) + ' Рейтинг: ' + str(t['Рейтинг']) + ' Лайки: ' +str(t['Лайки на отзыв']) 
                    + ' Дислайки:' + str(t['Дислайки на отзыв']) ) for index,t in df.iterrows()]
  return texts

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
texts = create_text_from_cols(train)
X_train, y_train, X_val, y_val = iterative_train_test_split((texts['text']).values.reshape(-1,1), np.array(train_labels), test_size=0.2)
test = pd.read_csv('Вологда/test_dataset_test.csv').fillna(0)
test_texts = create_text_from_cols(test)

In [ ]:
train_encodings = tokenizer([x[0] for x in X_train], truncation=True, padding=True,max_length=256)
val_encodings = tokenizer([x[0] for x in X_val], truncation=True, padding=True,max_length=256)
test_encodings = tokenizer(list(test_texts['text']), truncation=True, padding=True,max_length=256)


In [ ]:
import torch

class HHDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = np.array(labels,dtype = float)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).float()
        return item

    def __len__(self):
        return len(self.labels)




train_dataset = HHDataset(train_encodings, y_train)
val_dataset = HHDataset(val_encodings, y_val)
test_dataset = HHDataset(test_encodings, np.zeros((len(test_texts),5)))
#small_dataset = HHDataset(small_enc, y_val[:100,:])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=5, problem_type="multi_label_classification").to('cuda')

Some weights of the model checkpoint at sberbank-ai/ruBert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [ ]:
args = TrainingArguments(".", num_train_epochs=4,report_to='comet_ml',learning_rate = 1e-5,per_device_eval_batch_size=4,evaluation_strategy="steps",logging_strategy="steps",logging_steps = 400,save_steps = 2000, per_device_train_batch_size=4)
trainer = Trainer(model=model, args=args ,train_dataset=train_dataset, eval_dataset=val_dataset, tokenizer=tokenizer)


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3871
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3872
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_G

Step,Training Loss,Validation Loss
400,0.518000,0.438896
800,0.409400,0.360428
1200,0.322900,0.300724
1600,0.276400,0.251110
2000,0.233900,0.253503


***** Running Evaluation *****
  Num examples = 968
  Batch size = 4
***** Running Evaluation *****
  Num examples = 968
  Batch size = 4
***** Running Evaluation *****
  Num examples = 968
  Batch size = 4
***** Running Evaluation *****
  Num examples = 968
  Batch size = 4
***** Running Evaluation *****
  Num examples = 968
  Batch size = 4
Saving model checkpoint to ./checkpoint-2000
Configuration saved in ./checkpoint-2000/config.json
Model weights saved in ./checkpoint-2000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-2000/tokenizer_config.json
Special tokens file saved in ./checkpoint-2000/special_tokens_map.json


In [ ]:
preds = trainer.predict(test_dataset)

In [ ]:
probs = torch.sigmoid(torch.Tensor(preds.predictions))
pd.DataFrame(probs.numpy()).to_csv('roberta-preds.csv')

In [ ]:
answers = 1*((probs.numpy()>0.4))

In [ ]:
df_submission = pd.read_csv('Вологда/sample_solution.csv')

In [ ]:
df_submission.iloc[:,1:] = answers
df_submission.to_csv('answers-roberta.csv',index = False)